<img src="data/images/lecture-notebook-header.png" />

# Graph Mining: Community Detection

Community detection, also known as graph clustering or graph partitioning, is a technique in graph mining that aims to identify densely connected groups or communities of nodes within a graph. It involves dividing a graph into cohesive subgraphs where nodes within each subgraph have stronger connections among themselves compared to nodes in other subgraphs.

The goal of community detection is to uncover hidden structures and patterns within a network, revealing groups of nodes that exhibit similar characteristics, functionalities, or relationships. It has applications in various domains, including social network analysis, biological networks, recommendation systems, and web analysis.

Community detection algorithms employ various approaches to identify communities in a graph. Some common methods include:

* **Modularity-based methods:** These algorithms aim to optimize a quality function called modularity, which measures the strength of the community structure in a graph. The goal is to maximize the modularity value by iteratively merging or splitting nodes into communities.

* **Hierarchical methods:** These algorithms create a hierarchical structure of communities by iteratively merging or splitting nodes based on similarity measures or optimization criteria. They provide a multi-scale view of community structures, allowing identification of both global and local communities.

* **Spectral clustering:** This approach utilizes the eigenvectors or spectral properties of the graph's adjacency matrix to partition the graph into communities. It maps nodes to a lower-dimensional space and applies traditional clustering algorithms to identify communities.

* **Overlapping community detection:** Unlike traditional methods that assign each node to a single community, overlapping community detection algorithms allow nodes to belong to multiple communities. They identify overlapping regions in the graph where nodes participate in multiple communities simultaneously.

Community detection in graph mining provides valuable insights into the structure, function, and dynamics of networks. It helps in understanding the organization and behavior of complex systems, identifying influential nodes or communities, and uncovering hidden relationships and interactions. However, it is important to note that the notion of a "community" is subjective and can vary depending on the context and the specific goals of analysis. Therefore, the choice of a community detection algorithm should be aligned with the characteristics of the graph and the desired interpretation of communities.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Make all Required Imports

As an additional import, this notebook requires the [`python-louvain`](https://github.com/taynaud/python-louvain) package to be installed

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from networkx.algorithms import community
from networkx.algorithms.flow import minimum_cut

from collections import defaultdict

import community as louvain_community

---

## Generate Graph

### Define Graph Using Adjacency Matrix

There are many ways to define a graph with `networkX`. Here, we simply define the graph via its adjacency matrix, which is a basic 2-dimensional `numpy` array. As our example graph is unweighted, all entries in the adjacency matrix are 1.

In [ ]:
A = np.array([
    [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=float)

### Create Undirected Graph Using NetworkX

The `networkX` class `Graph` defines undirected graphs. We can use the adjacency matrix A to create an object of type `Graph` to represent our example graph. All the additional lines are only needed to specify the locations of the nodes when drawing the graph. This makes it easier to compare the graphs with each other, as all nodes will always be in the same position.

In [ ]:
G = nx.Graph(A)

## Define a position for each node
## (this not needed and only ensure that the graph looks like in the lecture)
fixed_positions = { 0:(0,0), 1:(-3,2), 2:(-6,0), 4:(-5,-3), 5:(-1,-3), 
                    3:(6,1), 7:(8,-2), 6:(9,-4),
                    8:(-1,-6), 9:(1, -9), 10:(1,-12), 12:(-3,-14), 14:(-6,-11), 15:(-5,-7),
                    11:(6,-9), 13:(6,-12) }

fixed_nodes = fixed_positions.keys()

pos = nx.spring_layout(G, pos=fixed_positions, fixed=fixed_nodes)

### Draw the Graph

`networkX` conveniently comes with in-built methods for that. The only thing that the method below adds is to color each node depending its belonging to a communities.

In [ ]:
def draw_communities(G, pos, communities):

    node_color = [0]*len(G.nodes)
    for idx, community in enumerate(communities):
        for node in community:
            node_color[node] = idx


    plt.figure()
    plt.axis('off')
    plt.tight_layout()
    nx.draw_networkx(G, pos, with_labels=True, font_weight='bold', node_color=node_color, cmap='cool')
    plt.show()
    
    
draw_communities(G, pos, (G.nodes, ))

In the example output above, we assume that all nodes for a single community; hence all nodes have the same color. We will re-use this drawing method in the following, if applicable.

---

## Girvan-Newman

The Girvan-Newman algorithm is a Divisive hierarchical approach. It starts with the whole graph representing a community and iteratively removes the "right" edges until the community is split into 2 sub-community. The "right" edges are the ones with the highest edge betweenness $c_{B}(e)$ for an edge $e$:

$$
c_{B}(e) = \sum_{v,w\in V} \frac{\sigma(v,w|e)}{\sigma(v,w)}
$$

where $\sigma(v,w|e)$ number of shortest paths from $v$ to $w$ going through edge $e$, and $\sigma(v,w)$ is the number of shortest paths from $v$ to $w$.

Simply speaking, an edge with a high edge betweenness value is an edge through which many shortest passes go through. This is generally true for edges that connect more tightly connected subgraphs (i.e., communities). This means that removing those edges first is the best way to split the whole graph into 2 communities. Of course The Girvan-Newman algorithm can then be applied again on the newly found communities to further split them into 2 sub-communities and so on.


### Running the Algorithm

The implementation of the Girvan-Newman algorithm provided by `networkX` calculates the communities on each hierarchical level and return an iterator over tuples of sets of nodes in G. Each set of node is a community, each tuple is a sequence of communities at a particular level of the algorithm.

As we've seen in the lecture, the algorithm is finished when each node represents its own community. In practice, this level of granularity is generally not meaningful. We there use a simple variable `max_depth` to indicate how far down the hierarchy we want to check the results

In [ ]:
## Feel free to change that
max_depth = 3

## Run Girvan-Newman Algorithm
communities_generator = community.girvan_newman(G)

for depth, communities in enumerate(communities_generator):
    
    ## Print the tuple with the communities
    print('Communities: {}'.format(communities))

    ## Draw the graph highlighting the communities
    ## (the output is conveniently already in the right format for our drawing method)
    draw_communities(G, pos, communities)
    
    ## Stop if we went down the hierarchiy far enough
    if depth+1 >= max_depth:
        break

**Note:** If you set `max_depth` rather high, the colors of the nodes might become indistinguishable. But you can always use the output of the print statement to check which nodes for a community on which level of the hierarchy.

---

## Louvain Algorithm

The Louvain algorithm for community detection is a method to extract communities from large graphs. The algorithm works by iteratively changing the assignment of nodes to communities in order to maximize the **modularity** if the graph. The algorithm is available as Python package: [`python-louvain`](https://python-louvain.readthedocs.io/en/latest/) which builds on `networkX`.


### Modularity Q

Modularity Q is the fraction of the edges that fall within the given groups minus the expected fraction if edges were distributed at random. The value of the modularity for unweighted and undirected graphs lies in the range $[-1/2,1]$ It is positive if the number of edges within groups exceeds the number expected on the basis of chance.

$$
Q = \frac{1}{2m}\sum_{vw} \left[ A[v,w] - \frac{k_vk_w}{2m}  \right ]\delta(c_v,c_w)
$$


where $m$ is the sum of weights of all edges, $A[v,w]$ is weight of edge between nodes $v$ and $w$, $k_i$ is the sum of the weights of edges attached to node $i$, $c_i$ is community of a node $i$, and

$$
\delta(c_v,c_w) = 
\begin{cases}
1 &, c_v=c_w \\
0 &, \text{otherwise}
\end{cases}
$$

is a simple indicator function that return $1$ if the two nodes $u$ and $v$ are in the same community

### Run Louvain Algorithm on G

As introduced in the lecture, the Louvain algorithm iteratively performs 2 steps:

* **Modularity Optimization:** For each node $v$, check if moving it to an adjacent community improves modularity $\Rightarrow$ move $v$ to the community that maximizes modularity
* **Graph Aggregation:** Represent each community as a new node and update the weights between new the nodes

The method `best_partition()` of package [`python-louvain`](https://python-louvain.readthedocs.io/en/latest/) perform but steps and return as dictionary with the nodes as keys and the id of the communities as value.

In [ ]:
best_partition_louvain = louvain_community.best_partition(G)

for node, community in best_partition_louvain.items():
    print('Node {} belongs to community {}'.format(node, community))

### Draw Communities

To use our method `draw_communities()` from above again we have to convert this dictionary into a valid input for the drawing method. This is all what the code cell below is doing.

In [ ]:
res = defaultdict(list)
for key, val in sorted(best_partition_louvain.items()):
    res[val].append(key)

communities = tuple(res.values())
    
print(communities)

Now we have everything in place to draw the communities.

In [ ]:
draw_communities(G, pos, communities)

### Further Partitioning

Of course, one can further partition the found communities further. We just iterate over each community and treat it as a subgraph to again run the Louvain algorithm on it.

In [ ]:
for i in range(len(communities)):
    
    G_sub = G.subgraph(communities[i])

    best_partition_louvain = louvain_community.best_partition(G_sub)
    
    print('Best partition for community', G_sub.nodes)
    for node, community in best_partition_louvain.items():
        print('\tNode {} belongs to community {}'.format(node, community))

Unfortunately, we cannot directly use `draw_communities()` as the number of nodes now has changed. However, the basic idea of further partitioning communities should be clear enough.

-----------------------------------------------

## Minimum Cut (Min-Cut)

A cut $C$ of a connected graph $G$ is a subset of the edges of G whose removal disconnects G.  That is, after removing all the edges of C, we can partition the vertices of G into two subsets $A$ and $B$ such that there are no edges between a vertex in $A$ and a vertex in $B$. A minimum cut of $G$ is a cut of smallest size among all cuts of $G$.

Intuitively, the subsets of vertices $A$ and $B$ can be interpreted as communities. In the following, we will test this intuition on our example graph to see if it results in (arguably) meaningful communities.

### Partition G Using Min-Cut

The following method splits an input graph $G$ into 2 subgraphs $A$ and $B$ by removing the Min-Cut from G, which by definition partitions the graph into two subgraphs. The method makes heavy use of methods provided by `networkX`.

In [ ]:
def min_cut_partition(G):
    
    ## If there is only one node, then there's nothing to be done
    if len(G.nodes) == 1:
        return None, None
    
    ## Create a copy of the graph so that we do not change the input graph itself
    G_copy = G.copy()
    
    ## Calculate the Min-Cut for the graph
    min_cut_edges = nx.minimum_edge_cut(G_copy)

    ## Remove all edges in the Min-Cut from the graph
    for e in min_cut_edges:
        G_copy.remove_edge(*e)
        
    ## Calculate the 2 components and sort them by size with the larger on being the first
    ## (we know that there can only be 2 components given the definition of Min-Cut)
    components = [ c for c in sorted(nx.connected_components(G_copy), key=len, reverse=True)]
    
    ## Create the 2 subgraphs A and B
    A = G_copy.subgraph(components[0])
    B = G_copy.subgraph(components[1])
        
    ## Return the size of the Min-Cut and the 2 subgraphs as a tuple
    return len(min_cut_edges), (A, B)

Using method `min_cut_partition()`, the following code iteratively splits the largest component/communities into 2 sub-components/sub-communities, starting with the original example graph $G$. In general, of course, both components after a partitioning can be further partitioned, but to keep the code simple we only do so for the largest one.

In [ ]:
## Start the algorithm with all the nodes
graph = G

while True:
    
    ## Partition the graph based on Min-Cut
    min_cut_size, partition = min_cut_partition(graph)
    
    ## If the partition is None (input graph had only 1 node), stop the loop
    if partition is None:
        break
        
    ## Pick the largest of the 2 component for further partitioning
    ## (we know the the larger component is first since we sort in the method about)
    graph = partition[0]
    
    print('|Min-Cut|={}, Largest component/community: {}'.format(min_cut_size, graph.nodes))

Given our idea of a community -- as well as compared to the results of the other algorithms -- the communities found based on Min-Cut do not look very useful. The fundamental problem is that subgraphs we would consider good communities just by eyeballing graph G are not "connected enough". The most obvious example is Node 15 which is the only node with only 1 edge. As such, the first Min-Cut is the set of edges containing only $(8, 15)$

In the next step, there are many nodes with only 2 edges, which all represent a value Min-Cut. Each case again results in a partition of 1 node and all the rest. And for our example graph, this continues to be so in each step: only 1 node gets partition off using Min-Cut. We can therefore argue that a Min-Cut-based approach is only suitable for very tightly connected communities and comparatively very few connections between communities.

---

## Summary

Community detection in graph mining is a powerful technique used to identify densely connected groups or communities within a graph. By uncovering hidden structures and patterns, it provides insights into the organization, behavior, and relationships of complex networks. However, community detection algorithms come with their own set of advantages and limitations.

One of the main advantages of community detection is its ability to reveal the underlying modular structure of a network. It helps in understanding the functional units or cohesive groups within a graph, which can be valuable for various applications such as social network analysis, recommendation systems, and biological network studies. Community detection allows researchers to gain insights into the behavior and dynamics of individual communities, enabling targeted analysis and interventions.

Another advantage is that community detection can uncover relationships and interactions that may not be apparent at the global level. By identifying communities, we can reveal local patterns and connections that might be overlooked when analyzing the graph as a whole. This can lead to the discovery of hidden communities or subgroups with distinct characteristics and behaviors, providing a more nuanced understanding of the network.

However, community detection also has its limitations. The notion of a "community" is subjective and can vary depending on the context and the specific goals of analysis. Different community detection algorithms may produce different results, making it challenging to determine the "correct" partitioning of a graph. Additionally, the scalability of community detection algorithms can be a concern for large-scale graphs, as some methods may become computationally expensive or impractical to apply.

Furthermore, community detection algorithms may struggle with networks that exhibit overlapping or hierarchical community structures. Traditional algorithms assign each node to a single community, which may not capture the complexity of real-world networks where nodes can belong to multiple communities or exist in nested communities. Developing effective algorithms for detecting and characterizing overlapping or hierarchical communities remains an active area of research.

In summary, community detection in graph mining offers valuable insights into network structures and relationships. It helps in understanding the organization and behavior of complex systems. While it has limitations such as subjective interpretation and scalability concerns, ongoing research aims to address these challenges and improve the accuracy and applicability of community detection algorithms.

In this notebook, we looked at 3 basic approaches to identify communities in a graph. But there are many more.